In [33]:
# ========= Cell 1: 填写任务信息 =========
task_name = "match_point_barrio"
dataset = "Bogota_points"
code_version = "v1.0"
input_dir = "E:/Dissertation/XGBoost_cleaning/2016central_Mask2Former_with_geo"     # 输入文件夹
output_dir = "E:/Dissertation/XGBoost_cleaning/2016central_Mask2Former_with_geobarrio"   # 输出文件夹
note = "每200个point生成一个batch，匹配Barrios社区+UPZ"
barrios_file = "E:/Dissertation/XGBoost_cleaning/Barrios_will_UPZ.geojson"

In [38]:
# ========= Cell 2: 批处理逻辑 =========
import os
import json
import time
import fiona
from shapely.geometry import shape, Point
from pathlib import Path

def batch_match_points(input_dir, output_dir, barrios_file, batch_size=200):
    start_time = time.time()
    os.makedirs(output_dir, exist_ok=True)

    # 读取社区边界
    barrios = []
    with fiona.open(barrios_file, "r") as src:
        for feat in src:
            geom = shape(feat["geometry"])
            props = feat["properties"]
            barrios.append((geom, props["barriocomu"], props["codigo_upz"]))

    # 筛选point文件
    all_files = [f for f in os.listdir(input_dir) if "point" in f and f.endswith(".json")]
    all_files.sort()

    batch_index = 1
    for i in range(0, len(all_files), batch_size):
        batch_files = all_files[i:i+batch_size]
        batch_folder = Path(output_dir) / f"batch_{batch_index:03d}"
        batch_folder.mkdir(parents=True, exist_ok=True)

        for filename in batch_files:
            file_path = Path(input_dir) / filename
            try:
                with open(file_path, "r", encoding="utf-8") as f:
                    data = json.load(f)
            except Exception as e:
                print(f"⚠️ 跳过坏文件: {filename}, 错误: {e}")
                continue

            lon, lat = data.get("longitude"), data.get("latitude")
            if lon is None or lat is None:
                print(f"⚠️ 文件缺少经纬度: {filename}")
                continue

            point = Point(lon, lat)

            barrio_name, upz_code = None, None
            for geom, bname, upz in barrios:
                if geom.contains(point):
                    barrio_name = bname
                    upz_code = upz
                    break

            data["barrio"] = barrio_name
            data["upz_code"] = upz_code

            out_path = batch_folder / filename
            with open(out_path, "w", encoding="utf-8") as f:
                json.dump(data, f, ensure_ascii=False, indent=2)


        print(f"完成 batch {batch_index:03d}，文件数 {len(batch_files)}")
        batch_index += 1

    duration = time.time() - start_time
    return duration


In [39]:
# ========= Cell 3: 写日志 =========
def append_log(task_name, dataset, code_version, input_dir, output_dir, status, duration, note):
    log_line = f"{task_name},{dataset},{code_version},{input_dir},{output_dir},{status},{duration:.2f},{note}\n"
    with open("task_log.csv", "a", encoding="utf-8") as f:
        f.write(log_line)

duration = batch_match_points(input_dir, output_dir, barrios_file, batch_size=200)
append_log(task_name, dataset, code_version, input_dir, output_dir, "done", duration, note)
print(f"任务完成，用时 {duration:.2f} 秒，日志已记录。")


⚠️ 文件缺少经纬度: point_0_panoramic_panoramic_primary_r01_20240310_121951_0950_007m_386519737567133_back.json
完成 batch 001，文件数 200
完成 batch 002，文件数 200
完成 batch 003，文件数 200
完成 batch 004，文件数 200
完成 batch 005，文件数 200
完成 batch 006，文件数 200
完成 batch 007，文件数 200
完成 batch 008，文件数 200
完成 batch 009，文件数 200
完成 batch 010，文件数 200
完成 batch 011，文件数 200
完成 batch 012，文件数 200
完成 batch 013，文件数 200
完成 batch 014，文件数 200
完成 batch 015，文件数 200
完成 batch 016，文件数 200
完成 batch 017，文件数 200
完成 batch 018，文件数 200
完成 batch 019，文件数 200
完成 batch 020，文件数 200
完成 batch 021，文件数 200
完成 batch 022，文件数 200
完成 batch 023，文件数 200
完成 batch 024，文件数 200
完成 batch 025，文件数 200
完成 batch 026，文件数 200
完成 batch 027，文件数 200
完成 batch 028，文件数 200
完成 batch 029，文件数 200
⚠️ 文件缺少经纬度: point_1560_back_four_direction_fallback_r02_20170926_014047_0457_999m_539223397077088.json
⚠️ 文件缺少经纬度: point_1560_front_four_direction_fallback_r01_20170926_014044_0668_012m_799363164345048.json
⚠️ 文件缺少经纬度: point_1560_right_four_direction_fallback_r03_20170926_014046_0907_00

In [45]:
# ===== 记录日志 =====
def append_log(task_name, dataset, code_version, input_dir, output_dir, status, duration, note):
    repo_dir = 'E:\Dissertation\CASA0004'
    log_path = f"{repo_dir}/operation_log.md"

    # 写入日志
    with open(log_path, "a", encoding="utf-8") as f:
        f.write(f"**任务名称**: {task_name}\n")
        f.write(f"**数据集**: {dataset}\n")
        f.write(f"**代码版本**: {code_version}\n")
        f.write(f"**输入目录**: {input_dir}\n")
        f.write(f"**输出目录**: {output_dir}\n")
        f.write(f"**状态**: {status}\n")
        f.write(f"**耗时**: {duration}\n")
        f.write(f"**备注**: {note}\n\n")

    print("✅ 日志写入完成")
status="finished"
append_log(task_name, dataset, code_version, input_dir, output_dir, status, duration, note)

✅ 日志写入完成


<>:3: SyntaxWarning: invalid escape sequence '\D'
<>:3: SyntaxWarning: invalid escape sequence '\D'
C:\Users\wengqc\AppData\Local\Temp\ipykernel_12392\4049532167.py:3: SyntaxWarning: invalid escape sequence '\D'
  repo_dir = 'E:\Dissertation\CASA0004'
